***
# 05. 벡터의 유사도(Vector Similarity)
***

문장이나 문서의 유사도를 구하는 작업은 자연어 처리의 주요 주제 중 하나입니다.
사람들이 인식하는 문서의 유사도는 주로 문서들 간에 동일한 단어 또는 비슷한 단어가 얼마나 공통적으로 많이 사용되었는지에 의존합니다.
기계도 마찬가지입니다. 기계가 계산하는 문서의 유사도의 성능은 각 문서의 단어들을 어떤 방법으로 수치화하여 표현했는지(DTM, Word2Vec 등),
문서 간의 단어들의 차이를 어떤 방법(유클리드 거리, 코사인 유사도 등)으로 계산했는지에 달려있습니다.

## 05-01 코사인 유사도(Cosine Similarity)
***

### 1. 코사인 유사도(Cosine Similarity)
***

**코사인 유사도**
- 두 벡터 간의 코사인 각도를 이용하여 구할 수 있는 두 벡터의 유사도를 의미
- 두 벡터의 방향이 완전히 동일한 경우에는 1의 값을 가지며, 90°의 각을 이루면 0, 180°로 반대의 방향을 가지면 -1의 값을 갖게 된다.
- 즉, 결국 코사인 유사도는 -1 이상 1 이하의 값을 가지며 값이 1에 가까울수록 유사도가 높다고 판단할 수 있다.
- 직관적으로 설명하면 두 벡터가 가리키는 방향이 얼마나 유사한가를 의미

![그림](img.png)


**두 벡터 A, B에 대해서 코사인 유사도의 식 표현**

![코사인 유사도 식](img_1.png)

문서 단어 행렬이나 TF-IDF 행렬을 통해서 문서의 유사도를 구하는 경우에는 문서 단어 행렬이나 TF-IDF 행렬이 각각의 특징 벡터 A, B가 된다.

예시를 통해 문서 단어 행렬에 대해서 코사인 유사도 구하기

문서1 : 저는 사과 좋아요
문서2 : 저는 바나나 좋아요
문서3 : 저는 바나나 좋아요 저는 바나나 좋아요

뛰어쓰기 기준 토큰화를 진행했다고 가정, 위의 세 문서에 대한 문서 단어 행렬

![문서 단어 행렬](img_2.png)

In [1]:
# Numpy를 사용해서 코사인 유사도를 계산하는 함수를 구현하고 각 문서 벡터 간의 코사인 유사도를 계산

import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

doc1 = np.array([0,1,1,1])
doc2 = np.array([1,0,1,1])
doc3 = np.array([2,0,2,2])

print('문서 1과 문서2의 유사도 :',cos_sim(doc1, doc2))
print('문서 1과 문서3의 유사도 :',cos_sim(doc1, doc3))
print('문서 2와 문서3의 유사도 :',cos_sim(doc2, doc3))

문서 1과 문서2의 유사도 : 0.6666666666666667
문서 1과 문서3의 유사도 : 0.6666666666666667
문서 2와 문서3의 유사도 : 1.0000000000000002


문서1과 문서2의 코사인 유사도와 문서1과 문서3의 코사인 유사도가 같고 문서2와 문서3의 코사인 유사도가 1이 나온다.
(1은 두 벡터의 방향이 완전히 동일한 경우에 1이 나오며, 코사인 유사도 관점에서는 유사도의 값이 최대임을 의미한다)

문서3은 문서2에서 단지 모든 단어의 빈도수가 1씩 증가했다.
즉, 한 문서 내의 모든 단어의 빈도수가 동일하게 증가하는 경우에는 기존의 문서와 코사인 유사도의 값이 1이다.

__EX)__
A 문서 : 요리 책, 50페이지 
B 문서 : 요리 책, 100페이지
C 문서 : 부동산 책, 50페이지

이런 경우 유클리드 거리로 유사도를 연산하면 문서 A가 문서 B보다 문서 C와 유사도가 더 높게 나오는 상황이 발생할 수 있다.
이는 유사도 연산에 문서의 길이가 영향을 받았기 때문인데, 이런 경우 코사인 유사도가 해결책이 될 수 있다.
코사인 유사도는 유사도를 구할 때 벡터의 방향(패턴)에 초점을 두므로 코사인 유사도는 문서의 길이가 다른 상황에서 비교적 공정한 비교를 할 수 있도록 도와준다.

### 2. 유사도를 이용한 추천 시스템 구현하기
***
캐글에서 사용되었던 영화 데이터셋으로 영화 추천 시스템 만들기
TF-IDF와 코사인 유사도만으로 영화의 줄거리에 기반해서 영화를 추천하는 추천 시스템을 만들 수 있다.

In [3]:
# 해당 데이터는 총 24개의 열을 가진 45,466개의 샘플로 구성된 영화 정보 데이터

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

data = pd.read_csv('movies_metadata.csv', low_memory=False)
data.head(2) # 다운로드 받은 훈련 데이터에서 상위 2개의 샘플만 출력하여 데이터의 형식을 확인
# 코사인 유사도에 사용할 데이터 : 영화 제목에 해당하는 title 열과 줄거리에 해당하는 overview 열

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


**좋아하는 영화를 입력하면, 해당 영화의 줄거리와 유사한 줄거리의 영화를 찾아서 추천하는 시스템을 만들기**

만약 훈련 데이터의 양을 줄이고 학습을 진행하고자 한다면 위와 같이 데이터를 줄여서 재저장할 수 있다.(상위 20,000개의 샘플만 사용)

In [5]:
# 상위 2만개의 샘플을 data에 저장
data = data.head(20000)

TF-IDF를 연산할 때 데이터에 Null 값이 들어있으면 에러가 발생
TF-IDF의 대상이 되는 data의 overview 열에 결측값에 해당하는 Null 값이 있는지 확인

In [6]:
# overview 열에 존재하는 모든 결측값을 전부 카운트하여 출력
print('overview 열의 결측값의 수:',data['overview'].isnull().sum()) # 135개의 Null 값

overview 열의 결측값의 수: 135


결측값을 가진 행을 제거하는 pandas의 dropna()나 결측값이 있던 행에 특정값으로 채워넣는 pandas의 fillna()를 사용할 수 있다.
괄호 안에 Null 대신 넣고자하는 값을 넣으면 된다. (실습에서는 빈 값(empty value)으로 대체)

In [7]:
# 결측값을 빈 값으로 대체
data['overview'] = data['overview'].fillna('') # Null 값을 빈 값으로 대체

overview열에 대해서 TF-IDF 행렬을 구한 후 행렬의 크기를 출력

In [9]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview'])
print('TF-IDF 행렬의 크기(shape) :',tfidf_matrix.shape) # 20,000의 행을 가지고 47,847의 열을 가지는 행렬

TF-IDF 행렬의 크기(shape) : (20000, 47487)


20,000개의 영화를 표현하기 위해서 총 47,487개의 단어가 사용됨 (47,847차원의 문서 벡터가 20,000개가 존재)
이제 20,000개의 문서 벡터에 대해서 상호 간의 코사인 유사도를 구합니다.

In [10]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과 :',cosine_sim.shape)

코사인 유사도 연산 결과 : (20000, 20000)


코사인 유사도 연산 결과로는 20,000행 20,000열의 행렬을 얻는다.
이는 20,000개의 각 문서 벡터(영화 줄거리 벡터)와 자기 자신을 포함한 20,000개의 문서 벡터 간의 유사도가 기록된 행렬입니다.
모든 20,000개 영화의 상호 유사도가 기록되어져 있습니다.


이제 기존 데이터프레임으로부터 영화의 타이틀을 key, 영화의 인덱스를 value로 하는 딕셔너리 title_to_index를 만들어둡니다.

In [56]:
title_to_index = dict(zip(data['title'], data.index))

# title_to_index 5개만 출력
title_dict = {}
for index, (key, value) in enumerate(title_to_index.items()):
    if index in range(5) :
        title_dict[key] = value
        
print(title_dict)

# 영화 제목 Father of the Bride Part II의 인덱스를 리턴
idx = title_to_index['Father of the Bride Part II']
print(idx)

{'Toy Story': 0, 'Jumanji': 1, 'Grumpier Old Men': 2, 'Waiting to Exhale': 3, 'Father of the Bride Part II': 4}
4


선택한 영화의 제목을 입력하면 코사인 유사도를 통해 가장 overview가 유사한 10개의 영화를 찾아내는 함수를 만듭니다.

In [73]:
def get_recommendations(title, cosine_sim=cosine_sim): # cosine_sim 기본값으로 전달
    # 선택한 영화의 타이틀로부터 해당 영화의 인덱스를 받아온다.
    idx = title_to_index[title]
    # 해당 영화와 모든 영화와의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx])) # [(title의 index, 유사도), (title의 index, 유사도), ...]

    # 유사도에 따라 영화들을 정렬한다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아온다.
    sim_scores = sim_scores[1:11] # [0]은 자기자신과의 유사도
    
    print(sim_scores[0]) # 자기자신과의 유사도
    print(idx) # 근데 왜 두 개의 인덱스가 다른지?
    
    # 가장 유사한 10개의 영화의 인덱스를 얻는다.
    movie_indices = [idx[0] for idx in sim_scores]
    print(sim_scores[0])
    print(movie_indices[0])
    # 가장 유사한 10개의 영화의 제목을 리턴한다.
    return data['title'].iloc[movie_indices] # 데이터에서 인덱스로 찾기


영화 다크 나이트 라이즈와 overview가 유사한 영화들을 찾아보겠습니다.

In [72]:
get_recommendations('The Dark Knight Rises')

(12481, 0.32152142350025487)
18252
(12481, 0.32152142350025487)
12481


12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object

가장 유사한 영화가 출력되는데, 영화 다크 나이트가 첫번째고, 그 외에도 전부 배트맨 영화를 찾아낸 것을 확인할 수 있습니다.

## 05-02 여러가지 유사도 기법
***

### 1. 유클리드 거리(Euclidean distance)
***

유클리드 거리(euclidean distance)는 문서의 유사도를 구할 때 자카드 유사도나 코사인 유사도만큼, 유용한 방법은 아니다.
하지만 여러 가지 방법을 이해하고, 시도해보는 것 자체만으로 다른 개념들을 이해할 때 도움이 되므로 의미가 있다.

다차원 공간에서 두개의 점 $p$와 $q$가 각각 $p=(p1, p2, p3, ..., pn)$과 $q=(q1, q2, q3, ..., qn)$의 좌표를 가질 때 두 점 사이의 거리를 계산하는 유클리드 거리 공식

![유클리드 거리 공식](img_3.png)

<br/>

2차원 공간이라고 가정하고 두 점 사이의 거리를 좌표 평면 상에서 시각화

![2차원시각화](img_4.png)

<br/>

2차원 좌표 평면 상에서 두 점 $p$와 $q$사이의 직선 거리를 구하는 문제
위의 경우에는 직각 삼각형으로 표현이 가능하므로, 중학교 수학 과정인 피타고라스의 정리를 통해 $p$와 $q$사이의 거리를 계산할 수 있다.
즉, 2차원 좌표 평면에서 두 점 사이의 유클리드 거리 공식은 피타고라스의 정리를 통해 두 점 사이의 거리를 구하는 것과 동일하다.

<br/>

여러 문서에 대해서 유사도를 구하고자 유클리드 거리 공식을 사용한다는 것은, 앞서 본 2차원을 단어의 총 개수만큼의 차원으로 확장하는 것과 같다.
예를 들어 아래와 같은 DTM이 있다고 하자

|     | 바나나 | 사과 | 저는 | 좋아요 |
|-----|-----|----|----|-----|
| 문서1 | 2   | 3  | 0  | 1   |
| 문서2 | 1   | 2  | 3  | 1   |
| 문서3 | 2   | 1  | 2  | 2   |

단어의 개수가 4개이므로, 이는 4차원 공간에 문서1, 문서2, 문서3을 배치하는 것과 같다.
이때 다음과 같은 문서Q에 대해서 문서1, 문서2, 문서3 중 가장 유사한 문서를 찾아내고자 한다.

|     | 바나나 | 사과 | 저는 | 좋아요 |
|-----|-----|----|----|-----|
| 문서Q | 1   | 1  | 0  | 1   |

이때 유클리드 거리를 통해 유사도를 구하려고 한다면, 문서Q 또한 다른 문서들처럼 4차원 공간에 배치시켰다는 관점에서 4차원 공간에서의 각각의 문서들과의 유클리드 거리를 구하면 된다.


파이썬 코드로 구현하기

In [1]:
import numpy as np

def dist(x,y):   
    return np.sqrt(np.sum((x-y)**2))

doc1 = np.array((2,3,0,1))
doc2 = np.array((1,2,3,1))
doc3 = np.array((2,1,2,2))
docQ = np.array((1,1,0,1))

print('문서1과 문서Q의 거리 :',dist(doc1,docQ)) # 가장 유사하다.
print('문서2과 문서Q의 거리 :',dist(doc2,docQ))
print('문서3과 문서Q의 거리 :',dist(doc3,docQ))


문서1과 문서Q의 거리 : 2.23606797749979
문서2과 문서Q의 거리 : 3.1622776601683795
문서3과 문서Q의 거리 : 2.449489742783178


유클리드 거리의 값이 가장 작다는 것은 문서 간 거리가 가장 가깝다는 것을 의미한다. 즉, 문서1이 문서Q와 가장 유사하다고 볼 수 있다.

### 2. 자카드 유사도(Jaccard similarity)
***

A와 B 두개의 집합이 있다고 합시다. 이때 교집합은 두 개의 집합에서 공통으로 가지고 있는 원소들의 집합을 말한다.
즉, 합집합에서 교집합의 비율을 구한다면 두 집합 A와 B의 유사도를 구할 수 있다는 것이 자카드 유사도(jaccard similarity)의 아이디어입니다.
자카드 유사도는 0과 1사이의 값을 가지며, 만약 두 집합이 동일하다면 1의 값을 가지고, 두 집합의 공통 원소가 없다면 0의 값을 갖습니다.
자카드 유사도를 구하는 함수를 $J$라고 하였을 때, 자카드 유사도 함수 $J$는 아래와 같습니다.

![유사도 함수](img_5.png)

<br/>

두 개의 비교할 문서를 각각 $doc1$, $doc2$라고 했을 때 $doc1$과 $doc2$의 문서의 유사도를 구하기 위한 자카드 유사도 : 

![자카드 유사도](img_6.png)

<br/>

두 문서 $doc1$, $doc2$ 사이의 자카드 유사도 $J(doc1, doc2)$는 두 집합의 교집합 크기를 두 집합의 합집합 크기로 나눈 값으로 정의된다.

In [2]:
doc1 = "apple banana everyone like likey watch card holder"
doc2 = "apple banana coupon passport love you"

# 토큰화
tokenized_doc1 = doc1.split()
tokenized_doc2 = doc2.split()

print('문서1 :',tokenized_doc1)
print('문서2 :',tokenized_doc2)

문서1 : ['apple', 'banana', 'everyone', 'like', 'likey', 'watch', 'card', 'holder']
문서2 : ['apple', 'banana', 'coupon', 'passport', 'love', 'you']


In [3]:
# 문서1과 문서2의 합집합 구하기

union = set(tokenized_doc1).union(set(tokenized_doc2))
print('문서1과 문서2의 합집합 :',union) # 문서1과 문서2의 합집합의 단어의 총 개수는 12개

문서1과 문서2의 합집합 : {'banana', 'you', 'love', 'passport', 'likey', 'coupon', 'everyone', 'apple', 'watch', 'like', 'holder', 'card'}


In [4]:
# 문서1과 문서2의 교집합 구하기 : 문서1과 문서2에서 둘 다 등장한 단어 찾기

intersection = set(tokenized_doc1).intersection(set(tokenized_doc2)) # 교집합 구하기
print('문서1과 문서2의 교집합 :',intersection) # 문서1과 문서2에서 둘 다 등장한 단어는 banana와 apple 총 2개

문서1과 문서2의 교집합 : {'banana', 'apple'}


In [5]:
# 교집합의 크기를 합집합의 크기로 나누면 자카드 유사도가 계산됨

print('자카드 유사도 :',len(intersection)/len(union))

자카드 유사도 : 0.16666666666666666
